In [1]:
import os,sys

# prerequisites
!{sys.executable} -m pip install ipywidgets
!{sys.executable} -m pip install ipycanvas
!{sys.executable} -m pip  install --upgrade OpenVisus
!jupyter.exe nbextension enable --py widgetsnbextension

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


Requirement already up-to-date: OpenVisus in c:\python38\lib\site-packages (2.1.106)


You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [2]:
import os,sys,math,random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import ipywidgets

from PIL            import Image
from urllib.request import urlopen
from ipycanvas      import Canvas, hold_canvas

from OpenVisus import *

"""
///////////////////////////////
CONFIG SECTION
///////////////////////////////
"""

dataset_url="http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1"


#'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern', 'gnuplot', 'gnuplot2', 'CMRmap', 
#'cubehelix', 'brg', 'gist_rainbow', 'rainbow', 'jet', 'nipy_spectral', 'gist_ncar'
cmap='gist_rainbow'
cmap='rainbow'
field='xray'
offset_step=4
canvas_width,canvas_height=600,600

Starting OpenVisus c:\python38\lib\site-packages\OpenVisus\__init__.py 3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)] sys.version_info(major=3, minor=8, micro=6, releaselevel='final', serial=0) ...


In [3]:
db=LoadDataset(dataset_url)
logic_box=db.getLogicBox()
dims=logic_box[1]

# /////////////////////////////////////////////////////////
def LoadImage(filename):
    return np.asarray(PIL.Image.open(urlopen(filename)  if filename.startswith('http') else filename))
    
# /////////////////////////////////////////////////////////
def PrintDatasetInfo():
    print("logic_box",logic_box)
    print("dims",dims)
    fields=[db.getField(name) for name in db.getFields()]
    for field in fields:
        dtype=field.dtype
        GB=int(dims[0]*dims[1]*dims[2]*dtype.getByteSize()/(1024*1024*1024))
        print("Field",field.name, dtype.toString(), "{}GB".format(GB))
    
# /////////////////////////////////////////////////////////
def ShowImage(data):
    m,M = np.min(data),np.max(data) 
    data=(data - m) / (M - m) # normalize 0,1
    data=plt.get_cmap(cmap)(data) # apply palette
    data = np.uint8(data*255) # convert uint8
    canvas.save()
    canvas.scale(canvas.width/data.shape[1],canvas.height/data.shape[0])
    canvas.put_image_data(data, 0, 0)
    canvas.restore()    
    
# /////////////////////////////////////////////////////////
def ExtractSlice(axis ,offset, num_refinements): 

    
    ranges=[(logic_box[0][I],logic_box[1][I]) for I in range(3)]
    
    x,y,z=(
        ranges[0] if axis!=0 else [offset,offset+1],
        ranges[1] if axis!=1 else [offset,offset+1],
        ranges[2] if axis!=2 else [offset,offset+1])

    # guess quality
    quality=0
    tot_pixels=(x[1]-x[0]) * (y[1]-y[0]) * (z[1]-z[0])
    MAX_PIXELS=canvas.width*canvas.height
    while tot_pixels >= MAX_PIXELS:
        quality-=1
        tot_pixels=int(tot_pixels/2) 

    print("Extracting slice","quality",quality,"num_refinements",num_refinements,"x",x,"y",y,"z",z)   
    
    while True:
        try:
            cont=0
            for data in db.read(x=x,y=y,z=z,num_refinements=num_refinements,quality=quality,field=field):
                # print("cont",cont,"{} {} quality({})".format(data.shape,data.dtype,quality));cont+=1
                if len(data.shape)==3 and data.shape[0]==1: data=data[0,:,:]
                if len(data.shape)==3 and data.shape[1]==1: data=data[:,0,:]   
                if len(data.shape)==3 and data.shape[2]==1: data=data[:,:,0] 
                ShowImage(data)
                print("{} {})".format(data.shape,data.dtype),end=' ')
            print("")
            break
        except  Exception as e:
            #it can fail because the quality is too low
            # print("increasing quality","error",e)
            quality+=1


# /////////////////////////////////////////////////////////
def ShowUI():
    
    axis_slider=ipywidgets.IntSlider(min=0, max=2, step=1, value=0)
    offset_slider=ipywidgets.IntSlider(min=0, max=dims[0]-1, step=offset_step, value=int(dims[0]/2))
    num_refinements_slider=ipywidgets.IntSlider(min=1, max=6, step=1, value=3)
    
    def OnAxisChange(*args):
        axis=axis_slider.value
        offset_slider.max=dims[axis]-1
        offset_slider.value=int(dims[axis]/2)       
    
    axis_slider.observe(OnAxisChange, 'value')
    #ipywidgets.interact_manual(ExtractSlice,  axis=axis_slider,  offset= offset_slider, num_refinements=num_refinements_slider)    
    ipywidgets.interact(ExtractSlice,  axis=axis_slider,  offset= offset_slider, num_refinements=num_refinements_slider)    

    
PrintDatasetInfo()
print("Add done")

logic_box ([0, 0, 0], [2048, 2048, 2048])
dims [2048, 2048, 2048]
Field DATA uint8 8GB
Add done


In [ ]:
canvas = Canvas(width=canvas_width, height=canvas_height);canvas

In [ ]:
ShowUI()